# TeAAL specifications on variants of Loops' SpMV implementations (Group-Mapped)

Description: Assign an equal amount of work tiles to a group of threads (warp or blocks). Threads within each group process individual work items in parallel.

In this version, each `tile` represents a row of $A$, and each `atom` represents a nonzero entry of $A$. Note that the `GRID_SIZE` (number of thread blocks) /= number of rows of $A$.

Template: https://github.com/gunrock/loops/blob/main/include/loops/algorithms/spmv/group_mapped.cuh

Scheduler (referenced as `config` below): https://github.com/gunrock/loops/blob/main/include/loops/schedule/group_mapped.hxx

GPU Kernel Template (Use it as a reference, don't execute it):

In [ ]:
__global__ void __launch_bounds__(threads_per_block, 2) __group_mapped(...) {
  # Initialize storage and schedule.
  using setup_t = schedule::block_mapped<threads_per_block, index_t, offset_t>; # Using block_mapped, assigning an entire tile (row of A) to a thread block
  using storage_t = typename setup_t::storage_t;
  __shared__ storage_t temporary_storage;

  # Construct the schedule.
  setup_t config(temporary_storage, offsets, A_rows, A_nnz);
  auto p = config.partition(); # Assigns work tiles to each thread block

  for (auto virtual_atom : config.atom_accessor(p)) { # Loop over total work, each thread processing individual work items
    auto virtual_tile = config.tile_accessor(virtual_atom, p);

    if (!(config.is_valid_accessor(virtual_tile, p)))
      continue;

    auto A_row = config.tile_id(virtual_tile, p); # Perform a binary-search to find the tile index.

    auto A_nz_idx = config.atom_id(virtual_atom, A_row, virtual_tile, p);
    atomicAdd(&(Z[A_row]), A_values[A_nz_idx] * B[indices[A_nz_idx]]);
  }
}

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "../..")

from src import utils

## Initialization

Initialize the input tensors.

For simplicity, the size of a thread warp is the same as the size of a thread block (`WARP_SIZE = BLOCK_SIZE`). Suppose that each GPU SM processes 1 thread warp/block per cycle.

In [ ]:
M = 8
K = 8

# GPU Kernel Configuration
BLOCK_SIZE = 2 # Number of threads per block
GRID_SIZE = (M + BLOCK_SIZE - 1) // BLOCK_SIZE # Number of thread blocks

print(f"GPU Kernel Configuration\n \
        BLOCK_SIZE (Number of threads per block): {BLOCK_SIZE} \n \
        GRID_SIZE (Number of thread blocks): {GRID_SIZE}")

# Calculating the number of work tiles per group
TILES_PER_GROUP = M // GRID_SIZE
print(f"TILES_PER_GROUP: {TILES_PER_GROUP}")

seed = 1

A_MK = Tensor.fromRandom(rank_ids=["M", "K"], shape=[M, K], seed=seed, density=[0.9, 0.6], name="A")
B_K = Tensor.fromRandom(rank_ids=["K"], shape=[K], seed=seed + 1, density=[1], name="B")

## TeAAL Specifications

Rows of matrix $A$ are partitioned across the SMs' warp/block. A thread warp/block can be assigned to a row with all zeros. 

Note that the current TeAAL specificaiton does not allow to specify the rank of `opt: slip`. This means there exists a synchronization across the SMs.

In [ ]:
yaml = """
einsum:
  declaration:
    A: [M, K]
    B: [K]
    Z: [M]
  expressions:
    - Z[m] = A[m, k] * B[k]
mapping:
  rank-order:
    A: [M, K]
    B: [K]
    Z: [M]
  partitioning:
    Z:
      M: [uniform_shape(TILES_PER_GROUP)]
      K: [uniform_occupancy(A.BLOCK_SIZE)]
  loop-order:
    Z: [M1, M0, K1, K0] 
    # M1: Number of partitioned rows of A = GRID_SIZE = Number of thread blocks
    # M0: Size of each partitioned row of A = TILES_PER_GROUP
    # K1: Number of partitioned nonzero elements for a given row
    # K0: Size of each partitioned nonzero elements = BLOCK_SIZE (Can be less than BLOCK_SIZE if there are less than BLOCK_SIZE nonzero elements left for the current partition) 
  spacetime:
    Z:
      space: [M1, K0]
      time: [M0, K1]
      # opt: slip # Turning off since currently not working as intended. Refer to the note above.
"""

utils.compile(yaml)

## Check Results (Correctness)

Check that the generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

In [ ]:
utils.check_matvecmul(A_MK, B_K, Z_M)

## Performance on GPU

Load Balance: Better load balance than Thread-Mapped, since rows with high NNZ will be processed by a group of threads (smoothing out heavy rows). Additionally, there's no need to worry about load balance across the warps since SMs can simply start processing on another thread warp when one warp finishes earlier than the others.

Assuming that the $A$ is stored in CSR format, $B$ and $Z$ are in uncompressed vectors, the memory access pattern would be:
- A: Coalesced access, threads in a warp are accessing the same row of $A$.
- B: Depends on the column indices of each nonzero entry of $A$. The more irregular the sparsity pattern that $A$ has, the more random the column indices of $A$'s nonzero entries will be. This should result in more uncoalesced accesses to $B$.
- Z: Coalesced access, threads in a warp are writing the same row of $Z$. At least with writing to $Z$, it may not be as fast as Thread-Mapped since it is done atomically on Group-Mapped to avoid data conflict. 